In [29]:
import pandas as pd
import asyncio
import websockets
import json
import uuid
import time
import requests

In [35]:
async def ping_relay(relay_url):
    rtt_ms = None
    try:
        start = time.time()
        async with websockets.connect(relay_url) as ws:
            end = time.time()
            rtt_ms = int((end - start) * 1000)
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(f"Error connecting to {relay_url}: {e}")
    return rtt_ms

def fetch_relay_info(url):
    headers = {'Accept': 'application/nostr+json'}
    data = None
    error = None
    relay_url = url[6:] if url.startswith('wss://') else url[5:] if url.startswith('ws://') else url
    try:
        response = requests.get(f"https://{relay_url}", headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
        else:
            error = response.text
    except Exception:
        try:
            response = requests.get(f"http://{relay_url}", headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
            else:
                error = response.text
        except Exception as e:
            error = str(e)
    return url, {'data': data, 'error': error}

rtt_ms = await ping_relay('wss://relay.damus.io')
url, info = fetch_relay_info('wss://relay.damus.io')

In [37]:
await ping_relay("wss://hcog4c6zjosd3soueyiitcg3jzq6ehwblles5n5snpv477unsrvz26qd.onion")

Error connecting to wss://hcog4c6zjosd3soueyiitcg3jzq6ehwblles5n5snpv477unsrvz26qd.onion: [Errno -2] Name or service not known


In [ ]:
relays = pd.read_csv('../data/raw/relays_url.csv')
for url in relays['url']:
    try:
        async with websockets.connect(url) as ws:
            print(f'Connected to {url}')
    except Exception as e:
        print(f'Failed to connect to {url}: {e}')
    finally:
        print('Done')

Checking wss://relay.damus.io
Connected to wss://relay.damus.io
Done
Checking wss://nostr-pub.wellorder.net
Connected to wss://nostr-pub.wellorder.net
Done
Checking wss://nostr.mom
Connected to wss://nostr.mom
Done
Checking wss://nostr.slothy.win
Failed to connect to wss://nostr.slothy.win: server rejected WebSocket connection: HTTP 403
Done
Checking wss://nostr.einundzwanzig.space
Connected to wss://nostr.einundzwanzig.space
Done
Checking wss://nos.lol
Connected to wss://nos.lol
Done
Checking wss://relay.nostr.band
Connected to wss://relay.nostr.band
Done
Checking wss://no.str.cr
Connected to wss://no.str.cr
Done
Checking wss://relay.cryptocculture.com
Done


CancelledError: 

In [ ]:
RELAY_URL = "wss://relay.nostrdice.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://relay.boroghor.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://schnorr.me"
RELAY_URL = "wss://mastodon.cloud/api/v1/streaming"
async def fetch_events(from_ts: int, to_ts: int):
    events = []
    subscription_id = str(uuid.uuid4())[:64]
    filter_obj = {
        # "since": from_ts,
        # "until": to_ts,
        # "limit": 10
    }

    async with websockets.connect(RELAY_URL) as ws:
        print(f"Connesso a {RELAY_URL}")

        req_msg = ["REQ", subscription_id, filter_obj]
        await ws.send(json.dumps(req_msg))
        print(f"Inviato: {req_msg}")

        try:
            while True:
                raw_message = await asyncio.wait_for(ws.recv(), timeout=30)
                message = json.loads(raw_message)

                if not isinstance(message, list):
                    print("Messaggio non valido (non è un array JSON)")
                    continue

                msg_type = message[0]

                if msg_type == "EVENT":
                    _, sub_id, event = message
                    if sub_id == subscription_id:
                        # print(f"Ricevuto evento: {event}")
                        events.append(event)

                elif msg_type == "EOSE":
                    _, sub_id = message
                    if sub_id == subscription_id:
                        print("Fine degli eventi storici.")
                        break

                elif msg_type == "NOTICE":
                    _, notice = message
                    print(f"NOTICE dal relay: {notice}")

                elif msg_type == "CLOSED":
                    _, sub_id, reason = message
                    if sub_id == subscription_id:
                        print(f"Subscription chiusa dal relay: {reason}")
                        break

                elif msg_type == "OK":
                    _, event_id, accepted, message_str = message
                    status = "accettato" if accepted else "rifiutato"
                    print(f"Evento {event_id} {status} - {message_str}")

                else:
                    print(f"Messaggio sconosciuto: {message}")

        except asyncio.TimeoutError:
            print("Timeout: nessun messaggio ricevuto per 30 secondi.")
        finally:
            close_msg = ["CLOSE", subscription_id]
            await ws.send(json.dumps(close_msg))
            print("Subscription chiusa.")
    return events

# Esegui in una cella async
async def main():
    to_time = int(time.time())
    from_time = to_time - 600
    return await fetch_events(0, 1727090175)

events = await main()


InvalidStatus: server rejected WebSocket connection: HTTP 403